## User guide
https://pcse.readthedocs.io/en/stable/user_guide.html#getting-started


In [ ]:
import pcse
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pp

## Running PCSE/WOFOST with custom input data
For running PCSE/WOFOST (and PCSE models in general) with your own data sources you need three different types of inputs:

1. Model parameters that parameterize the different model components. These parameters usually consist of :
   - a set of crop parameters (or multiple sets in case of crop rotations), 
   - a set of soil parameters and 
   - a set of site parameters: ancillary parameters that are specific for a location.

2. Driving variables represented by weather data which can be derived from various sources.
3. Agromanagement actions which specify the farm activities that will take place on the field that is simulated by PCSE.

In [ ]:
from pcse.fileinput import CABOFileReader, YAMLCropDataProvider
from pathlib import Path
from nevergrad.functions.irrigation.common_path import IRRIGATION_DATA_DIR

## 1. Model parameters
### 1.1. Crop parameters

In [ ]:
cropfile = Path(IRRIGATION_DATA_DIR, "crop")
cropdata = YAMLCropDataProvider(fpath=cropfile)

In [ ]:
pp(cropdata.get_crops_varieties())

### 1.2. Soil parameters
- soil type 
- soil physical properties

In [ ]:
soilfile = Path(IRRIGATION_DATA_DIR, "soil", "ec3.soil") 
# we will use the water balance for 
# freely draining soils and use the soil file for medium fine sand
soildata = CABOFileReader(soilfile)

In [ ]:
pp(soildata)

### 1.3. Site parameters

In [ ]:
from pcse.util import WOFOST71SiteDataProvider

In [ ]:
# the initial conditions of the water balance such as the initial soil moisture content
sitedata = WOFOST71SiteDataProvider(
    WAV=100, # initial soil moisture content
    CO2=360, # the atmospheric CO2 concentration
)

In [ ]:
expected_sitedata = {
    'IFUNRN': 0,
    'NOTINF': 0,
    'SSI': 0.0, # the initial surface storage
    'SSMAX': 0.0, # the maximum surface storage
    'WAV': 100.0,
    'SMLIM': 0.4
}
assert sitedata == expected_sitedata

### 1.4. Combine them

In [ ]:
from pcse.base import ParameterProvider

In [ ]:
parameters = ParameterProvider(cropdata=cropdata, soildata=soildata, sitedata=sitedata)

## 2. Weather data

In [ ]:
from pcse.db import NASAPowerWeatherDataProvider
from nevergrad.functions.irrigation.irrigation import KNOWN_GEOLOCS

In [ ]:
address = "Lome"
geoloc = KNOWN_GEOLOCS.get(address)

In [ ]:
wdp = NASAPowerWeatherDataProvider(latitude=geoloc.get("latitude"), longitude=geoloc.get("longitude"))

In [ ]:
print(wdp)

## 3. Agromanagement

https://github.com/ajwdewit/pcse_notebooks/blob/master/06_advanced_agromanagement_with_PCSE.ipynb

In [ ]:
from pcse.fileinput import YAMLAgroManagementReader

In [ ]:
agromanagement_file = Path(IRRIGATION_DATA_DIR, "agromanagement", 'rice_schedule.agro')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
print(agromanagement)

## 4. Initializing WOFOST model

In [ ]:
from pcse.models import Wofost72_WLP_FD

In [ ]:
wofost_sim = Wofost72_WLP_FD(parameters, wdp, agromanagement)

In [ ]:
wofost_sim.run_till_terminate()

In [ ]:
outputs = wofost_sim.get_output()
df_output = pd.DataFrame(outputs)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 5))
var = "LAI"
df_output.set_index("day")[var].plot(ax=ax, ylabel=var)